In [1]:
from langchain_aws import ChatBedrock
import dotenv
import os

dotenv.load_dotenv()
# Set LANGSMITH_PROJECT environment variable
os.environ['LANGSMITH_PROJECT'] = "agent_playground"


# Claude is a good alternative to GPT-4o:
# https://blog.promptlayer.com/big-differences-claude-3-5-vs-gpt-4o/
llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    # model_id="us.meta.llama3-3-70b-instruct-v1:0",
    # model_id="us.meta.llama3-2-3b-instruct-v1:0",
    # model_id="us.deepseek.r1-v1:0",
    model_kwargs=dict(temperature=0.2),
    # other params...
)

# Supervisor-style Agent

In [2]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool

# Create specialized agents

@tool
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

@tool
def web_search(query: str) -> str:
    """Search the web for information."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )


math_agent = create_react_agent(
    model=llm,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

web_search_agent = create_react_agent(
    model=llm,
    tools=[web_search],
    name="web_search_expert",
    prompt="You are a web search expert. Do not do anything else other than web search."
)

# Create supervisor workflow
workflow = create_supervisor(
    [web_search_agent, math_agent],
    model=llm,
    prompt=(
        "You are a team supervisor managing a web search expert and a math expert. "
        "For current events, use web_search_expert. "
        "For math problems, use math_agent."
    )
)

# Compile and run
app = workflow.compile()

In [3]:
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what's the combined headcount of the FAANG companies in 2024?"
        }
    ]
})
result

Failed to get info from https://api.smith.langchain.com: LangSmithConnectionError('Connection error caused failure to GET /info in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError(\'HTTPSConnectionPool(host=\\\'api.smith.langchain.com\\\', port=443): Max retries exceeded with url: /info (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x128b5d850>: Failed to resolve \\\'api.smith.langchain.com\\\' ([Errno 8] nodename nor servname provided, or not known)"))\'))\nContent-Length: None\nAPI Key: lsv2_********************************************e9')
Error raised by bedrock service
Traceback (most recent call last):
  File "/Users/simd/wc_simd/.conda/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/simd/wc_simd/.conda/lib/python3.11/site-packages/urllib3/util/connection.py", line 60, in create_connection


EndpointConnectionError: Could not connect to the endpoint URL: "https://portal.sso.eu-west-1.amazonaws.com/federation/credentials?role_name=weco-developer&account_id=760097843905"

Failed to batch ingest runs: langsmith.utils.LangSmithConnectionError: Connection error caused failure to POST https://api.smith.langchain.com/runs/batch in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/batch (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x13c1a75d0>: Failed to resolve \'api.smith.langchain.com\' ([Errno 8] nodename nor servname provided, or not known)"))'))
Content-Length: 25735
API Key: lsv2_********************************************e9
post: trace=4afb5f6f-7384-473d-b748-e13c82a78285,id=4afb5f6f-7384-473d-b748-e13c82a78285; trace=4afb5f6f-7384-473d-b748-e13c82a78285,id=12d47dde-fb08-4931-81c5-488b5c2f90e6; trace=4afb5f6f-7384-473d-b748-e13c82a78285,id=bbef694a-7e3a-4ad4-9363-c819578db7f8; trace=4afb5f6f-7384-473d-b748-e13c82a78285,id=d03d6525-affe-4f0a-90a8-e6a403a255dc; trace=4afb5f6f-7384-47